<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/ninologo.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

---

To get started: consult [start](start.ipynb)

---

# Search Introduction

*Search* in Text-Fabric is a template based way of looking for structural patterns in your dataset.

It is inspired by the idea of
[topographic query](http://books.google.nl/books?id=9ggOBRz1dO4C).

Within Text-Fabric we have the unique possibility to combine the ease of formulating search templates for
complicated patterns with the power of programmatically processing the results.

This notebook will show you how to get up and running.

## Alternative for hand-coding

Search is a powerful feature for a wide range of purposes.

Quite a bit of the implementation work has been dedicated to optimize performance.
Yet I do not pretend to have found optimal strategies for all
possible search templates.
Some search tasks may turn out to be somewhat costly or even very costly.

That being said, I think search might turn out helpful in many cases,
especially by reducing the amount of hand-coding needed to work with special subsets of your data.

## Easy command

Search is as simple as saying (just an example)

```python
results = A.search(template)
A.show(results)
```

See all ins and outs in the
[search template docs](https://annotation.github.io/text-fabric/tf/about/searchusage.html).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use

In [3]:
A = use("Nino-cunei/ninmed:clone", checkout="clone", hoist=globals())
# A = use("Nino-cunei/ninmed", hoist=globals())

This is Text-Fabric 9.2.5
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

45 features found and 0 ignored


# Basic search command

We start with the most simple form of issuing a query.
Let's look for the numerals with a repeat greater than 3.
We also want to show the words in which they occur.

All work involved in searching takes place under the hood.

In [4]:
query = """
word
  sign type=numeral
"""
results = A.search(query)

  0.08s 958 results


In [5]:
A.table(results, end=10)

n,p,word,sign
1,P285136 obverse:2',5,5
2,P285136 obverse:11',2,2
3,P479250 reverse:16,7,7
4,P479250 reverse:16,7-šu₂,7
5,P479250 reverse:16,7-šu₂,7
6,P285109 obverse:1:5',13,1
7,P285109 obverse:1:5',13,3
8,P285109 obverse:1:5',10,10
9,P285109 obverse:1:6',UD.3.KAM₂# [,3
10,P285109 obverse:1:7',10,10


The hyperlinks take us all to the CDLI archival page of the document (tablet) in question.

Note that we can choose start and/or end points in the results list.

In [6]:
A.table(results, start=500, end=503)

n,p,word,sign
500,P394520 reverse:2:1,7,7
501,P394520 reverse:2:1,7,7
502,P394523 obverse:1:3',10,10
503,P394523 obverse:1:4',UD.3.KAM@v,3


We can show the results more fully with `show()`.

That gives us pretty displays of tablet lines with the results highlighted.

In [7]:
A.show(results, end=3)

# Condense results

There are two fundamentally different ways of presenting the results: condensed and uncondensed.

In **uncondensed** view, all results are listed individually.
You can keep track of which parts belong to which results.
The display can become unwieldy.

This is the default view, because it is the straightest, most logical, answer to your query.

In **condensed** view all nodes of all results are grouped in containers first (e.g. verses), and then presented
container by container.
You loose the information of what parts belong to what result.

As an example of is the difference, we look for all numerals.

In [8]:
query = """
% we choose a tablet with several numerals
document pnumber=P285109
  sign type=numeral
"""

Note that you can have comments in a search template. Comment lines start with a `%`.

In [9]:
results = A.search(query)
A.table(results, end=10)

  0.07s 9 results


n,p,document,sign
1,P285109 obverse:1:5',P285109,1
2,P285109 obverse:1:5',P285109,3
3,P285109 obverse:1:5',P285109,10
4,P285109 obverse:1:6',P285109,3
5,P285109 obverse:1:7',P285109,10
6,P285109 obverse:1:9',P285109,5
7,P285109 obverse:1:11',P285109,1
8,P285109 obverse:1:11',P285109,3
9,P285109 obverse:1:12',P285109,3


Let's expand the results display:

In [10]:
A.show(results, end=2, skipCols="1")

As you see, the results are listed per result tuple, even if they occur all in the same verse.
This way you can keep track of what exactly belongs to each result.

Now in condensed mode:

In [11]:
A.show(results, condensed=True, withNodes=True)

The last line has two results, and both results are highlighted in the same line display.

We can modify the container in which we see our results.

By default, it is `line`, but we can make it `face` as well:

In [12]:
A.show(results, end=2, condensed=True, condenseType="face")

We now see the the displays of two faces, one with two numerals in it and one with three.

# Custom highlighting

Let us make a new search where we look for two different things in the same line.

We can apply different highlight colors to different parts of the result.
The signs in the pair are member 0 and 1 of the result tuples.
The members that we do not map, will not be highlighted.
The members that we map to the empty string will be highlighted with the default color.

**NB:** Choose your colors from the
[CSS specification](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value).

In [13]:
query = """
line
  sign missing=1
  sign question=1
  sign damage=1
"""

In [14]:
results = A.search(query)
A.table(results, end=10, baseTypes="sign")

  0.23s 21274 results


n,p,line,sign,sign,sign
1,P479250 obverse:13',[... ] ŠA₃?# GIG,...,ŠA₃?#,ŠA₃?#
2,P479250 obverse:13',[... ] ŠA₃?# GIG,],ŠA₃?#,ŠA₃?#
3,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,...,pi?#,pi?#
4,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,ap,pi?#,pi?#
5,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,-,pi?#,pi?#
6,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,],pi?#,pi?#
7,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,...,SAHAR?#,SAHAR?#
8,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,...,SAHAR?#,TA#
9,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,...,SAHAR?#,urudu#
10,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,...,SAHAR?#,na#


In [15]:
A.table(
    results,
    end=10,
    baseTypes="sign",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

n,p,line,sign,sign,sign
1,P479250 obverse:13',[... ] ŠA₃?# GIG,...,ŠA₃?#,ŠA₃?#
2,P479250 obverse:13',[... ] ŠA₃?# GIG,],ŠA₃?#,ŠA₃?#
3,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,...,pi?#,pi?#
4,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,ap,pi?#,pi?#
5,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,-,pi?#,pi?#
6,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,],pi?#,pi?#
7,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,...,SAHAR?#,SAHAR?#
8,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,...,SAHAR?#,TA#
9,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,...,SAHAR?#,urudu#
10,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,...,SAHAR?#,na#


In [16]:
A.show(results, end=5, colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"})

Color mapping works best for uncondensed results. If you condense results, some nodes may occupy
different positions in different results. It is unpredictable which color will be used
for such nodes:

In [17]:
A.show(
    results,
    condensed=True,
    end=5,
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

You can specify to what container you want to condense. By default, everything is condensed to lines.

Let's change that to faces.
Note that the `end` parameter counts the number of faces now.

In [21]:
A.show(
    results,
    start=4,
    end=6,
    condensed=True,
    condenseType="line",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

# Constraining order
You can stipulate an order on the things in your template.
You only have to put a relational operator between them.
Say we want only results where the damage follows the missing.

In [22]:
query = """
line
  sign question=1
  sign missing=1
  < sign damage=1
"""

In [23]:
results = A.search(query)
A.table(results, end=10, baseTypes="sign")

  0.22s 11608 results


n,p,line,sign,sign,sign
1,P479250 obverse:13',[... ] ŠA₃?# GIG,ŠA₃?#,...,ŠA₃?#
2,P479250 obverse:13',[... ] ŠA₃?# GIG,ŠA₃?#,],ŠA₃?#
3,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,pi?#,...,pi?#
4,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,pi?#,ap,pi?#
5,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,pi?#,-,pi?#
6,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,pi?#,],pi?#
7,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,SAHAR?#,...,SAHAR?#
8,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,SAHAR?#,...,TA#
9,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,SAHAR?#,...,urudu#
10,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,SAHAR?#,...,na#


We can also require the things to be adjacent.

In [24]:
query = """
line
  sign question=1
  sign missing=1
  <: sign damage=1
"""

In [26]:
results = A.search(query)
A.table(results, end=10, baseTypes="sign")
A.show(
    results,
    end=2,
    baseTypes="sign",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

  0.18s 575 results


n,p,line,sign,sign,sign
1,P479250 obverse:13',[... ] ŠA₃?# GIG,ŠA₃?#,],ŠA₃?#
2,P479250 reverse:16,[... ap-] pi?# šil-ta-hi ša₂ 7 KA.KEŠDA-MEŠ-šu₂ 7-šu₂ u 7-šu₂,pi?#,],pi?#
3,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,SAHAR?#,],SAHAR?#
4,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,SAHAR?#,],urudu#
5,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,bi?#,],SAHAR?#
6,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,bi?#,],urudu#
7,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,ana?,],SAHAR?#
8,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,ana?,],urudu#
9,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,bi?#,],SAHAR?#
10,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,bi?#,],urudu#


Finally, we make the three things fully adjacent in fixed order:

In [27]:
query = """
line
  sign question=1
  <: sign missing=1
  <: sign damage=1
"""

In [29]:
results = A.search(query)
A.table(results, end=10, baseTypes="sign")
A.show(results, end=4, colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"})

  0.19s 31 results


n,p,line,sign,sign,sign
1,P479250 reverse:17,[... ] SAHAR?# TA# urudu#na#-ah#-bi?# [ana? ] urudu#na#-ah#-bi?#,ana?,],urudu#
2,P394104 obverse:2:28,[... A?] u₂#IN₆.UŠ₂ A u₂EME.UR.GI₇ A u₂GEŠTIN.KA₅.A,A?,],u₂#
3,P394104 obverse:2:37,[tu?-gal?-lab?] u₂#BAL# NUMUN# U₂# [da-] aq#-qi₂-qi₂ ŠE₁₀ up-pu-te gu-ra ša₂ GI NUMUN u₂ta-lu-pa-di,lab?,],u₂#
4,P393782 obverse:1:22,u₂IGI-NIŠ SUHUŠ gišNAM.TAR NITA₂ a-he-en#-[na-a GAZ? SIM? ina ] ŠA₃?# mal#-ma#-liš# TI#-qe₂# ina# KAŠ?# [NAG? ina A? ] ŠED₇?#,A?,],ŠED₇?#
5,P393782 obverse:1:34,[NAG-MEŠ 2?] UD#-me# ina A-MEŠ NU pa-tan NAG-MEŠ A-MEŠ SAG ŠA₃-šu₂ tu#-sa#[-lah₂-ma TI],2?,],UD#
6,P393782 obverse:2:10,DIŠ NA ŠA₃-šu₂ ka-si-šu₂ SUMsar SIG₇-su šimGUR₂#[.GUR₂? ] TEŠ₂#.BI SUD₂ ina# A SIG₃-aṣ NU pa-tan NAG-šu₂,GUR₂?,],TEŠ₂#
7,P393782 obverse:2:21,[EN₂ ŠA₃-MEŠ? ] it#-ta#-nap-ša₂-ru ze-er kar-šu₂# ŠA₃.NIGIN ku-uṣ-ṣu#-ru#-u₂?# x x x [x],MEŠ?,],it#
8,P393782 obverse:2:44,1# GIN₂ MUN eme-sal-li₃ šim#[GUR₂.GUR₂?] šim#LI DIŠ-niš GAZ [SIM?],GUR₂?,],šim#
9,P393782 reverse:1:10,a#-[na NENNI? A? NENNI? ] DU₁₁#.GA#-ma iṣ-ṣa-bat ŠA₃-bi NENNI?# [A? NENNI? x x x x x x x x x x x x x x x x],NENNI?,],DU₁₁#
10,P365742 obverse:1:54,13 SILA₃ PA gišMA₂#.[ERIŠ₄?] iš#.MA₂.RA GAZ SIM ina A GAZIsar SILA₁₁-aš SAR#-ab# LAL#-[ma] UD.3.KAM₂ NU DU₈,ERIŠ₄?,],iš#


# Custom feature display

We would like to see the original atf and the flags for signs.
The way to do that, is to perform a `A.prettySetup(features)` first.

We concentrate on one specific result.

In [30]:
A.displaySetup(extraFeatures="atf flags")

In [32]:
A.show(
    results,
    start=3,
    end=3,
    baseTypes="sign",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

The features without meaningful values have been left out. We can also change that by passing a set of values
we think are not meaningful. The default set is

```python
{None, 'NA', 'none', 'unknown'}
```

In [34]:
A.displaySetup(noneValues=set())
A.show(
    results,
    start=3,
    end=3,
    baseTypes="sign",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

This makes clear that it is convenient to keep `None` in the `noneValues`:

In [35]:
A.displaySetup(noneValues={None})
A.show(
    results,
    start=3,
    end=3,
    baseTypes="sign",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

We can even choose to suppress other values, e.g. the value 1.

That will remove all the features such as `question`, `missing`.

In [36]:
A.displaySetup(noneValues={None, "NA", "unknown", 1})
A.show(
    results,
    start=3,
    end=3,
    baseTypes="sign",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

In the rest of the notebook we stick to our normal setup, so we reset the extra features.

In [38]:
A.displayReset()
A.show(
    results,
    start=3,
    end=3,
    baseTypes="sign",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

# Features from queries

In earlier displays we saw the *types* of signs, because the query mentioned it.

Suppose we want to display the type also here, then we can modify the query by mentioning the feature `type`.

But we do not want to impose extra limitations, so we say `type*`, meaning: no conditions on type whatsoever.

In [39]:
query = """
line
  sign question=1 type*
  <: sign missing=1
  <: sign damage=1
"""

In [41]:
results = A.search(query)
A.show(
    results, start=3, end=3, colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"}
)

  0.18s 31 results


We do not see the features, because they are `sign` features, and our display stops at the `word` level.
But we can improve on that:

In [42]:
A.show(
    results,
    start=3,
    end=3,
    baseTypes="sign",
    colorMap={0: "", 2: "cyan", 3: "magenta", 4: "lightsalmon"},
)

# Show your own tuples

So far we have `show()`n the results of searches.
But you can also construct your own tuples and show them.

Whereas you can use search to get a pretty good approximation of what you want, most of the times
you do not arrive precisely at your destination.

Here is an example where we use search to come close, and then work our way to produce the end result.

## More missing than damaged

We look for lines that have more missing signs than damaged signs.

In our search templates we cannot formulate that a feature has different values on two nodes in the template.
We could spell out all possible combinations of values and make a search template for each of them,
but that is needlessly complex.

Let's first use search to find all clauses containing missing and damaged signs.

In [43]:
query = """
line
  sign missing
  sign damage
"""
results = A.search(query)

  0.21s 47648 results


Now the hand coding begins. We are going to extract the tuples we want.

In [44]:
lines = {}
for (l, m, d) in results:
    lines.setdefault(l, (set(), set()))
    lines[l][0].add(m)
    lines[l][1].add(d)
print(f"{len(lines)} lines")

2182 lines


Now we have all lines with both missing and damaged signs, without duplicates.

For each line we have a set with its missing signs and one with its damaged signs.

We filter in order to retain the lines with more missing than damaged signs.
We put all missing signs in one big set and all damaged signs in one big set.

In [45]:
answer = []
missing = set()
damage = set()

for (l, (m, d)) in lines.items():
    if len(m) > len(d):
        answer.append((l, *m, *d))
        missing |= m
        damage |= d
len(answer)

1676

In [46]:
answer[0]

(85972, 3, 4, 5, 6, 7, 8, 1)

We are going to make a dictionary of highligts: one color for the missing signs and one for the damaged.

In [47]:
highlights = {}
colorM = "lightsalmon"
colorD = "mediumaquamarine"
for s in missing:
    highlights[s] = colorM
for s in damage:
    highlights[s] = colorD

And now we can show them:

In [49]:
A.table(answer, start=1, end=5, highlights=highlights)

As you see, you have total control.

---

All chapters:

* **[start](start.ipynb)** become an expert in creating pretty displays of your text structures
* **[display](display.ipynb)** become an expert in creating pretty displays of your text structures
* **search** turbo charge your hand-coding with search templates
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **[similarLines](similarLines.ipynb)** spot the similarities between lines

---

See the [cookbook](cookbook) for recipes for small, concrete tasks.

CC-BY Dirk Roorda